## o4 Refactor

In [1]:
import os
import geopandas as gp    #used for handling geospatial Census Tract data
import pandas as pd
from scipy.stats import norm    #used to compute earthquake damage probability
import numpy as np
import time
import matplotlib as plt


# Import Damage Function Variables Spreadsheet

# The damage function variable table explained:

 Each row represents a combination of building type and building code
 building type refers to the (such as structure or materials of the building: frames, wall types)
 building codes are the classification of its seismic codes
 for example: HC (high code) is the most resistant. 

 Each column (median moderate, median extensive, median complete ...) refers to a damage state
 The values represents the PGA (Peak Ground Acceleration) value at which such damage state will occur.
 For example, if MedianModerate = 0.22, it means given the building type and building code,
moderate damage can be expected at a PGA of 0.22.

Those columns were followed by the beta columns (BetaSlight, BetaModerate, BetaExtensive...)
Those are lognormal standard deviations (used to compute confidence intervals or model uncertainty).
Small beta means this certain type of building has similar falling threshold (smaller uncertainty)

In [2]:
#dmgfvars = r"..\Tables\DamageFunctionVariables.csv"
parent_dir = os.path.dirname(os.getcwd())
dmgfvars = os.path.join(parent_dir, "Tables", "DamageFunctionVariables.csv")
#dmgfvars = r"DamageFunctionVariables.csv"
dmgfvarsDF = pd.read_csv(dmgfvars)
dmgfvarsDF = dmgfvarsDF.drop('Unnamed: 0', axis=1)
list_bldgtypes = dmgfvarsDF["BLDG_TYPE"].unique()

# Extract median cols, beta cols, and damage levels
# List of cols with emdian estimates
median_columns = [col for col in dmgfvarsDF.columns if col.lower().startswith('median')]

# list of beta columns
beta_columns = [col for col in dmgfvarsDF.columns if col.lower().startswith('beta')]

# list of the levels - used for renaming variables 
pga_levels = ['slight', 'mod', 'ext', 'comp']


In [3]:
# ASSUMPTION 1:  
# Each building type (W1, W2 etc..) has multiple relevant rows for different seismic codes
# When available, we're keeping the highest code (HC). If that's not available, we're keeping MC etc..

# Define priority order for BUILDINGCO
priority_order = {"HC": 1, "MC": 2, "LC": 3, "PC": 4}

# Sort by BLDG_TYPE and priority of BUILDINGCO
dmgfvarsDF["priority"] = dmgfvarsDF["BUILDINGCO"].map(priority_order)
dmgfvarsDF = dmgfvarsDF.sort_values(["BLDG_TYPE", "priority"])

# Keep only the first occurrence of each BLDG_TYPE (i.e., highest priority)
dmgfvars_hc = dmgfvarsDF.groupby("BLDG_TYPE").first().reset_index().drop(columns=["priority"])

## Import O3 results

In [4]:
building_types_o3 = ['W1', 'W2', 'S1L', 'S1M', 'S1H', 'S2L', 'S2M', 'S2H', 'S3', 'S4L', 'S4M', 'S4H', 'S5L', 'S5M', 'S5H', 'C1L', 'C1M', 'C1H', 'C2L', 'C2M', 'C2H', 'C3L', 'C3M', 'C3H', 'PC1', 'PC2L', 'PC2M', 'PC2H', 'RM1L', 'RM1M', 'RM2L', 'RM2M', 'RM2H', 'URML', 'URMM', 'MH']  # Get all the structure type columns
dtypes = {}
for bldg_type in building_types_o3:
    dtypes[bldg_type] = 'float64'
dtypes

{'W1': 'float64',
 'W2': 'float64',
 'S1L': 'float64',
 'S1M': 'float64',
 'S1H': 'float64',
 'S2L': 'float64',
 'S2M': 'float64',
 'S2H': 'float64',
 'S3': 'float64',
 'S4L': 'float64',
 'S4M': 'float64',
 'S4H': 'float64',
 'S5L': 'float64',
 'S5M': 'float64',
 'S5H': 'float64',
 'C1L': 'float64',
 'C1M': 'float64',
 'C1H': 'float64',
 'C2L': 'float64',
 'C2M': 'float64',
 'C2H': 'float64',
 'C3L': 'float64',
 'C3M': 'float64',
 'C3H': 'float64',
 'PC1': 'float64',
 'PC2L': 'float64',
 'PC2M': 'float64',
 'PC2H': 'float64',
 'RM1L': 'float64',
 'RM1M': 'float64',
 'RM2L': 'float64',
 'RM2M': 'float64',
 'RM2H': 'float64',
 'URML': 'float64',
 'URMM': 'float64',
 'MH': 'float64'}

In [5]:
eventid = 'nc72282711'
GPKG_PATH = os.path.join(parent_dir, "Shakemaps", eventid, "eqmodel_outputs.gpkg")
o3_results = gp.read_file(GPKG_PATH, layer='tract_shakemap_pga')
#o3_results = pd.read_csv('o3_output_nc72282711.csv')
o3_results

,GEOID,max_intensity,min_intensity,mean_intensity,OTHER_OTHER,RESIDENTIAL_MULTI FAMILY,RESIDENTIAL_OTHER,RESIDENTIAL_SINGLE FAMILY,TOTAL_RESIDENTIAL,TOTAL_BUILDING,...,RM1L,RM1M,RM2L,RM2M,RM2H,URML,URMM,MH,Total,geometry
0,06001400100,0.02,0.02,0.02,104.0,22.0,10.0,1162.0,1194.0,1298.0,...,0.021052631578947368,0.0,0.0014035087719298245,0.0,0.0,0.0035087719298245615,0.0,0.0,1.0007017543859649,"MULTIPOLYGON (((-122.24692 37.88544, -122.2466..."
1,06001400200,0.02,0.02,0.02,46.0,110.0,2.0,538.0,650.0,696.0,...,0.0323785803237858,0.0,0.0024906600249066002,0.0,0.0,0.0062266500622665,0.0,0.0,1.0024906600249066,"MULTIPOLYGON (((-122.25792 37.84261, -122.2577..."
2,06001400300,0.02,0.02,0.02,67.0,416.0,7.0,1139.0,1562.0,1629.0,...,0.03692674210839786,0.0,0.0023823704586063135,0.0,0.0,0.008933889219773675,0.0,0.0,1.0000000000000002,"MULTIPOLYGON (((-122.26563 37.83764, -122.2655..."
3,06001400400,0.02,0.02,0.02,57.0,391.0,3.0,777.0,1171.0,1228.0,...,0.03163191948238677,0.0,0.0014378145219266715,0.0,0.0,0.007189072609633357,0.0,0.0,0.9985621854780734,"MULTIPOLYGON (((-122.26183 37.84162, -122.2618..."
4,06001400500,0.02,0.02,0.02,56.0,342.0,6.0,614.0,962.0,1018.0,...,0.03521779425393883,0.0,0.0018535681186283596,0.0,0.0,0.009267840593141797,0.0,0.0,1.0009267840593143,"MULTIPOLYGON (((-122.26951 37.84858, -122.2693..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2600,06115040800,0.01,0.01,0.01,454.0,128.0,136.0,1256.0,1520.0,1974.0,...,0.024025974025974027,0.0,0.001948051948051948,0.0,0.0,0.005844155844155844,0.0,0.04935064935064935,1.0006493506493503,"MULTIPOLYGON (((-121.51553 39.03064, -121.5153..."
2601,06115040901,0.01,0.01,0.01,388.0,109.0,491.0,1025.0,1625.0,2013.0,...,0.016681299385425813,0.0,0.001755926251097454,0.0,0.0,0.003511852502194908,0.0,0.3520632133450395,1.0008779631255487,"MULTIPOLYGON (((-121.58338 39.13621, -121.5831..."
2602,06115040902,0.01,0.01,0.01,759.0,0.0,0.0,0.0,0.0,759.0,...,0.03128911138923655,0.0,0.0025031289111389237,0.0,0.0,0.007509386733416771,0.0,0.0050062578222778474,0.997496871088861,"MULTIPOLYGON (((-121.47722 39.13334, -121.4770..."
2603,06115041000,0.01,0.01,0.01,759.0,0.0,0.0,0.0,0.0,759.0,...,0.03128911138923655,0.0,0.0025031289111389237,0.0,0.0,0.007509386733416771,0.0,0.0050062578222778474,0.997496871088861,"MULTIPOLYGON (((-121.63637 39.24608, -121.6362..."


In [6]:
# change data type of columns
o3_results = o3_results.astype(dtypes)

In [7]:
## ASSUMPTIONS

# In the absence of a TOTAL column i will assume that adding all the categories leads to a total

o3_results['Total_Num_Building'] = o3_results['OTHER_OTHER'] + o3_results['RESIDENTIAL_MULTI FAMILY'] + o3_results['RESIDENTIAL_OTHER']+ o3_results['RESIDENTIAL_SINGLE FAMILY']
col = o3_results.pop('Total_Num_Building')  #re-order
o3_results.insert(6, 'Total_Num_Building', col)
o3_results.head()

,GEOID,max_intensity,min_intensity,mean_intensity,OTHER_OTHER,RESIDENTIAL_MULTI FAMILY,Total_Num_Building,RESIDENTIAL_OTHER,RESIDENTIAL_SINGLE FAMILY,TOTAL_RESIDENTIAL,...,RM1L,RM1M,RM2L,RM2M,RM2H,URML,URMM,MH,Total,geometry
0,06001400100,0.02,0.02,0.02,104.0,22.0,1298.0,10.0,1162.0,1194.0,...,0.021053,0.0,0.001404,0.0,0.0,0.003509,0.0,0.0,1.0007017543859649,"MULTIPOLYGON (((-122.24692 37.88544, -122.2466..."
1,06001400200,0.02,0.02,0.02,46.0,110.0,696.0,2.0,538.0,650.0,...,0.032379,0.0,0.002491,0.0,0.0,0.006227,0.0,0.0,1.0024906600249066,"MULTIPOLYGON (((-122.25792 37.84261, -122.2577..."
2,06001400300,0.02,0.02,0.02,67.0,416.0,1629.0,7.0,1139.0,1562.0,...,0.036927,0.0,0.002382,0.0,0.0,0.008934,0.0,0.0,1.0000000000000002,"MULTIPOLYGON (((-122.26563 37.83764, -122.2655..."
3,06001400400,0.02,0.02,0.02,57.0,391.0,1228.0,3.0,777.0,1171.0,...,0.031632,0.0,0.001438,0.0,0.0,0.007189,0.0,0.0,0.9985621854780734,"MULTIPOLYGON (((-122.26183 37.84162, -122.2618..."
4,06001400500,0.02,0.02,0.02,56.0,342.0,1018.0,6.0,614.0,962.0,...,0.035218,0.0,0.001854,0.0,0.0,0.009268,0.0,0.0,1.0009267840593143,"MULTIPOLYGON (((-122.26951 37.84858, -122.2693..."


In [8]:
# ALSO IN THE ABSENCE OF PROPER COLUMNS FROM O3 --> I will multiply these here
# Multiply total buildings by percentage columns to estimate counts
building_types_o3 = ['W1', 'W2', 'S1L', 'S1M', 'S1H', 'S2L', 'S2M', 'S2H', 'S3', 'S4L', 'S4M', 'S4H', 'S5L', 'S5M', 'S5H', 'C1L', 'C1M', 'C1H', 'C2L', 'C2M', 'C2H', 'C3L', 'C3M', 'C3H', 'PC1', 'PC2L', 'PC2M', 'PC2H', 'RM1L', 'RM1M', 'RM2L', 'RM2M', 'RM2H', 'URML', 'URMM', 'MH']  # Get all the structure type columns

o3_results[building_types_o3] = o3_results[building_types_o3].multiply(o3_results['Total_Num_Building'], axis=0)

In [9]:
# Only keep relevant columns
# for the sake of conveninence in this first pass

o3_results = o3_results[['GEOID',
 'max_intensity',
 'min_intensity',
 'mean_intensity',
 'geometry',
 'Total_Num_Building', 'W1', 'W2', 'S1L', 'S1M', 'S1H', 'S2L', 'S2M', 'S2H', 'S3', 'S4L', 'S4M', 'S4H', 
 'S5L', 'S5M', 'S5H', 'C1L', 'C1M', 'C1H', 'C2L', 'C2M', 'C2H', 'C3L', 'C3M', 'C3H', 'PC1', 'PC2L', 'PC2M', 'PC2H', 
 'RM1L', 'RM1M', 'RM2L', 'RM2M', 'RM2H', 'URML', 'URMM', 'MH']]




## Step 1: Calculate the probability of each type of damage per building structure

#### Goal: In this section, we would like to know what is the probability of each type of damage (slight, mod, compl, extensive) given the type of structure and the min PGA for that tract

In [10]:
# Initialize a dictionary to store computed probabilities
prob_dict = {}

# Loop through each building type

#TODO: For future, add a check that all buildings in list_bldgtypes exist in o3 results. Otherwise errors might occur
for bldg_type in list_bldgtypes:

    # Loop through each PGA level
    for i in range(len(pga_levels)):

        # Extract Beta for the current building type
        beta = dmgfvars_hc.loc[dmgfvars_hc['BLDG_TYPE'] == bldg_type, beta_columns[i]].item() #TODO: this assumes that the list of betas is ranked by damage level. Update this to use ilike to ensure prooper betas are being passed
        # Extract the correct median PGA threshold
        PGA_median = dmgfvars_hc.loc[dmgfvars_hc['BLDG_TYPE'] == bldg_type, median_columns[i]].item() #TODO: this assumes that the list of MEDIANS is ranked by damage level. Update this to use ilike to ensure prooper betas are being passed

       # Compute probability for the current building type and damage level
        prob_dict[f'P_{pga_levels[i].lower()}_{bldg_type}'] = norm.cdf((1 / beta) * np.log(o3_results['min_intensity'] / PGA_median))

# Add all computed probabilities to once
df = pd.concat([o3_results, pd.DataFrame(prob_dict)], axis=1)


## STEP 2: Estimate cumulative  number of buildings with certain levels of damage

## STEP 3: Get number of building for each category (subtract off the amount counted under other categories).

In [11]:
# Initialize dictionaries to store computed values
num_slight, num_moderate, num_extensive, num_complete = {}, {}, {}, {} # cumulative counts
ex_slight, ex_moderate, ex_extensive, ex_complete = {}, {}, {}, {} # Exclusive counts (non-cumulative)

# Loop through each building type to compute damage estimates
for bldg_type in list_bldgtypes:
    # Cumulative damage estimates
    num_slight[f'numSlight_{bldg_type}'] = df[bldg_type] * df[f'P_slight_{bldg_type}']
    num_moderate[f'numModerate_{bldg_type}'] = num_slight[f'numSlight_{bldg_type}'] * df[f'P_mod_{bldg_type}']
    num_extensive[f'numExtensive_{bldg_type}'] = num_moderate[f'numModerate_{bldg_type}'] * df[f'P_ext_{bldg_type}']
    num_complete[f'numComplete_{bldg_type}'] = num_extensive[f'numExtensive_{bldg_type}'] * df[f'P_comp_{bldg_type}']

    # Exclusive damage estimates
    ex_slight[f'numSlight_{bldg_type}'] = num_slight[f'numSlight_{bldg_type}'] - num_moderate[f'numModerate_{bldg_type}']
    ex_moderate[f'numModerate_{bldg_type}'] = num_moderate[f'numModerate_{bldg_type}'] - num_extensive[f'numExtensive_{bldg_type}']
    ex_extensive[f'numExtensive_{bldg_type}'] = num_extensive[f'numExtensive_{bldg_type}'] - num_complete[f'numComplete_{bldg_type}']
    ex_complete[f'numComplete_{bldg_type}'] = num_complete[f'numComplete_{bldg_type}']

# Assign all computed columns to df at once
df = pd.concat([df, pd.DataFrame({**ex_slight, **ex_moderate, **ex_extensive, **ex_complete})], axis=1)

## STEP 4: Get the total number of buildings in each damage category 

In [12]:
## STEP 4: Get the total number of buildings in each damage category 

df["Total_Num_Building_Slight"] = df.filter(like="numSlight").sum(axis=1)
df["Total_Num_Building_Moderate"] = df.filter(like="numModerate").sum(axis=1)
df["Total_Num_Building_Extensive"] = df.filter(like="numExtensive").sum(axis=1)
df["Total_Num_Building_Complete"] = df.filter(like="numComplete").sum(axis=1)

In [13]:
df_final = df[['GEOID', 'max_intensity', 'min_intensity','mean_intensity', 'geometry',
               'Total_Num_Building', 'Total_Num_Building_Slight', 'Total_Num_Building_Moderate', 
               'Total_Num_Building_Extensive', 'Total_Num_Building_Complete']]

df_final.head()

,GEOID,max_intensity,min_intensity,mean_intensity,geometry,Total_Num_Building,Total_Num_Building_Slight,Total_Num_Building_Moderate,Total_Num_Building_Extensive,Total_Num_Building_Complete
0,06001400100,0.02,0.02,0.02,"MULTIPOLYGON (((-122.24692 37.88544, -122.2466...",1298.0,0.049786,9.548432e-07,6.362047e-12,3.061619e-18
1,06001400200,0.02,0.02,0.02,"MULTIPOLYGON (((-122.25792 37.84261, -122.2577...",696.0,0.033236,1.824007e-06,3.413057e-11,2.210400e-17
2,06001400300,0.02,0.02,0.02,"MULTIPOLYGON (((-122.26563 37.83764, -122.2655...",1629.0,0.083951,5.051363e-06,8.318287e-11,5.274653e-17
3,06001400400,0.02,0.02,0.02,"MULTIPOLYGON (((-122.26183 37.84162, -122.2618...",1228.0,0.059529,3.696162e-06,6.952613e-11,4.502756e-17
4,06001400500,0.02,0.02,0.02,"MULTIPOLYGON (((-122.26951 37.84858, -122.2693...",1018.0,0.054835,3.967408e-06,7.430542e-11,4.812087e-17
